# Predice el futuro

###### Importing all required libaries.
<blockquote>numpy is the fundamental package for scientific computing with Python.</blockquote>
<blockquote>pandas it provides ready to use high-performance data structures and data analysis tools</blockquote>
<blockquote>ARIMA it provides the capability to fit an ARIMA model.</blockquote>

In [26]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARIMA

Load the data by running the following code.

In [27]:
data_1 = pd.read_csv('train_csv.csv')
avg= data_1['feature']
avg=list(avg)

below creating one-dimensional ndarray of feature values and time with given format.

In [28]:
res = pd.Series(avg, index=pd.to_datetime(data_1['time'], format='%Y-%m-%d %H:%M:%S'))

Natural log of the column is computed using log() function and stored in a new column namely “ts”  as shown below. In second line of code taking difference between ts and  ts.shift(). here shift() - Shift index by desired number of periods with an optional time freq. Below code giving differences.

In [29]:
ts=np.log(res)
ts_diff = ts - ts.shift()

below used dropna() function to remove missing values with inplace attribute true. it is do operation inplace and return none.

In [30]:
ts_diff.dropna(inplace=True)

Here below fit an ARIMA model to the time column in train dataset and review the residual errors.

First, I have fit an ARIMA(2,1,2) model. This sets the lag value to 2 for autoregression, uses a difference order of 1 to make the time series stationary, and uses a moving average model of 2.

When fitting the model, a lot of debug information is provided about the fit of the linear regression model. We can turn this off by setting the disp argument to -1.

In [31]:
r = ARIMA(ts,(2,1,2))
r = r.fit(disp=-1)

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency 10S will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency 10S will be used.
  % freq, ValueWarning)


This can be passed to the predict() function on our model in order to predict the class values for each instance in the array with start and end attributes. And The date_range() function is used to get a fixed frequency DatetimeIndex with frequency 10 seconds beacuse of in test dataset it is incremented by 10 seconds.

In [32]:
pred = r.predict(start='2019-03-19 00:13:20',end='2019-03-19 00:19:50')
dates = pd.date_range('2019-03-19 00:13:20','2019-03-19 00:19:50',freq='10S')

Below here again used series function and it is returning array object. 

The cumsum function returning Series of the same size containing the cumulative sum.

Again used series function to get array object in line 3. After that using add function we can doing addition of dataframe with fill_value attribute 0.

Then we are calculating exponential of all the elements in the input array. It shown in below.

In [33]:
predictions_ARIMA_diff = pd.Series(pred, copy=True)
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
predictions_ARIMA_log = pd.Series(ts[:])
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
predictions_ARIMA = np.exp(predictions_ARIMA_log)
print(predictions_ARIMA)

2019-03-19 00:00:00    237.970000
2019-03-19 00:00:10     57.930000
2019-03-19 00:00:20     90.530000
2019-03-19 00:00:30    260.750000
2019-03-19 00:00:40     49.700000
                          ...    
2019-03-19 00:19:10      1.768428
2019-03-19 00:19:20      1.782928
2019-03-19 00:19:30      1.797543
2019-03-19 00:19:40      1.812274
2019-03-19 00:19:50      1.827123
Length: 120, dtype: float64


The final_pedict dictionary is created to write id and feature value from above arrays. For feature value here I have adding prediction value with mean of difference between perdicted values of seconds which are in predictions_ARIMA.head().

In [34]:
# create dict for writing output
final_pedict ={'id': [], 'feature': []}
k=0
number = []
feature = []
for i in range(80, 120):
    if k == 5:
        k=0
    if k  < 5:
        number.append(i)
        feature.append(predictions_ARIMA_log[i]+predictions_ARIMA.head()[k])
        k = k + 1

final_pedict = {'id': number, 'feature': feature}
df = pd.DataFrame(final_pedict)


Below In df object we retured id and perdicted feature values. And using this object we are writing csv file with file name 'Solution.csv' with no indexing.

In [35]:
df.to_csv('Solution.csv',index=False)
print("Created Successfully")

Created Successfully
